# Preparação do ambiente

## Bibliotecas

In [102]:
import json
import pandas as pd
import re

## Funções auxiliares

In [2]:
NULL_STRING = '#NULLO#'

In [3]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [25]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [44]:
def extract_magalu_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

# Carga e prepação dos dados

## Amazon

In [28]:
file_amazon_reindexed = '../datasets/amazon_reindexed.parquet'
df_amazon = pd.read_parquet(file_amazon_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Descrição_Principal', 'Imagem', 'Imagens' ]
df_amazon = df_amazon[columns_to_keep]

df_amazon['desc'] = df_amazon[['Descrição', 'Descrição_Principal']].apply(lambda row: extract_amazon_desc(row),axis=1)
# df_amazon['img_list'] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)
df_amazon[['img0','img1','img2']] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)

columns_to_keep = ['Link', 'Nome', 'desc', 'img0','img1','img2']
df_amazon = df_amazon[columns_to_keep]
df_amazon.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']
df_amazon.head()

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


## Magalu

In [45]:
file_magalu_reindexed = '../datasets/magalu_redindexed.parquet.gzip'
df_magalu = pd.read_parquet(file_magalu_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_magalu = df_magalu[columns_to_keep]

df_magalu[['img0','img1','img2']] = df_magalu[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_magalu = df_magalu[columns_to_keep]
df_magalu.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_magalu.sample(5)

,link,nome,desc,img0,img1,img2
54996,https://www.magazineluiza.com.br/smartphone-re...,Smartphone Redm 12c Dual Sim 128gb - Verde - C...,Smartphone de 128gb 4 de ramTela de 6.71Leitor...,https://a-static.mlcdn.com.br/90x90/smartphone...,,
27055,https://www.magazineluiza.com.br/campainha-sma...,Campainha Smart Com Camera De Video Sem Fio Wi-Fi,campainha inteligente sem fio,https://a-static.mlcdn.com.br/90x90/campainha-...,,
46185,https://www.magazineluiza.com.br/mini-ar-condi...,Mini Ar Condicionado Portátil - VALECOM,Mini Ar Condicionado Portátil Para Quarto Idea...,https://a-static.mlcdn.com.br/280x210/mini-ar-...,https://a-static.mlcdn.com.br/90x90/mini-ar-co...,https://a-static.mlcdn.com.br/90x90/mini-ar-co...
38447,https://www.magazineluiza.com.br/fone-de-ouvid...,Fone de Ouvido Bluetooth ANC 10M Preto - BRIGHT,"Item indispensavel para quem busca conforto, p...",https://a-static.mlcdn.com.br/90x90/fone-de-ou...,https://a-static.mlcdn.com.br/90x90/fone-de-ou...,https://a-static.mlcdn.com.br/90x90/fone-de-ou...
1914,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/caixa-de...,,


## Mercado Livre

In [56]:
file_ml_reindexed = '../datasets/ml_redindexed.parquet.gzip'
df_ml = pd.read_parquet(file_ml_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_ml = df_ml[columns_to_keep]

df_ml[['img0','img1','img2']] = df_ml[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_ml = df_ml[columns_to_keep]
df_ml.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_ml.sample(5)

,link,nome,desc,img0,img1,img2
76011,https://www.mercadolivre.com.br/xiaomi-redmi-n...,Xiaomi Redmi Note 11 Pro 5g White 6gbram Globa...,Câmera dupla e mais detalhes Suas 2 câmeras tr...,https://http2.mlstatic.com/D_NQ_NP_774475-MLU6...,,
50118,https://produto.mercadolivre.com.br/MLB-387204...,Battria Compatível iPhone XS Larcado 2658 Mah,(PRODUTO COMPATÍVEL COM O MODELO INFORMADO) A ...,https://http2.mlstatic.com/D_NQ_NP_682659-MLB7...,,
30773,https://produto.mercadolivre.com.br/MLB-336804...,Smartphone Motorola Moto E6i Dual Sim 32 Gb Pi...,"Somos a Fixcomm, a maior do Brasil em recondic...",https://http2.mlstatic.com/D_NQ_NP_718357-MLB7...,,
59161,https://produto.mercadolivre.com.br/MLB-418369...,Smartphone LG G4 Stylus Hdtv 16gb 1gb Ram Bege...,"Olá. Seja bem vindo a nossa loja, A Shopex é u...",https://http2.mlstatic.com/D_NQ_NP_964050-MLB7...,https://http2.mlstatic.com/D_NQ_NP_926894-MLB7...,
53361,https://produto.mercadolivre.com.br/MLB-403004...,Amplificador De Reforço De Sinal De Antena Par...,Características: Adota-se ABS+cobre puro+silic...,https://http2.mlstatic.com/D_NQ_NP_983918-CBT7...,,


# União dos dados em único dataframe

In [92]:
df_anuncios = pd.concat([df_amazon, df_magalu, df_ml])
df_anuncios = df_anuncios.reset_index(drop=True)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


In [93]:
df_anuncios['link'].apply(lambda x: re.search('[a-z]+(?=.com)',x).group(0)).unique()

array(['amazon', 'perfdrive', 'magazineluiza', 'mercadolivre'],
      dtype=object)

# Extratir amostra

In [101]:
n_samples = 20000
samples = df_anuncios.iloc[:,1:].sample(n_samples,random_state=111).to_dict('records')

In [103]:
file_samples = '../datasets/samples_to_label.json'
with open(file_samples,'w') as f:
    json.dump(samples,f)

array(['amazon', 'perfdrive', 'magazineluiza', 'mercadolivre'],
      dtype=object)